# Capstone Project - The Battle of the Neighborhoods (Sushi Restaurants in the city of Toronto) : Code
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Importing Packages, Sourcing Data and Structuring the Dataframe](#introduction)
* [Adding Location Coordinates](#coordinates)
* [Getting Venue Data using Foursqure API](#api)
* [Identifying the unique characteristics of each neighbourhood](#characteristics)
* [Using K-Means to Cluster Neighbourhoods](#k-means)
* [Plotting the Clustered Neighbourhoods](#clusterplot)


<h1> Importing Packages, Sourcing Data and Structuring the Table </h1><a name="introduction"></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
%pip install folium
import folium

     |████████████████████████████████| 102kB 165kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [27]:
Wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(Wiki_url).text

In [28]:
soup = BeautifulSoup(source, 'xml')


In [29]:
table=soup.find('table')


In [30]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)


In [31]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [32]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [33]:
df=df[df['Borough']!='Not assigned']
df


,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [34]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [35]:
df_merge = pd.merge(df, temp_df, on='Postalcode')


In [36]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)


In [37]:
df_merge.drop_duplicates(inplace=True)


In [38]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)
df_merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [39]:
df_merge.shape


(103, 3)

<h1> Adding Location Coordinates </h1> <a name="coordinates"></a>

In [40]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude


In [41]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')


In [42]:
geo_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [44]:
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]


In [45]:
geo_data.head()


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [46]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [47]:
print(toronto_data.groupby('Borough').count()['Neighbourhood'])


Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64


In [48]:
#df.groupby('Borough').count()['Neighbourhood']


In [49]:
boroughs = toronto_data['Borough'].unique().tolist()
#creating of list of unique boroughs in toronto to be used later for visualizations

In [50]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [51]:
lat_toronto = toronto_data['Latitude'].mean()
lon_toronto = toronto_data['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [52]:
from IPython.display import display
base_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'],
                                           toronto_data['Borough'], 
                                           toronto_data['Neighbourhood']):
    label_text = borough + ' - ' + neighbourhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(base_toronto)  
    
base_toronto

# Getting Venue data using the Foursquare API <a name="api"></a>

In [53]:
CLIENT_ID = '0ZZFVO4RE4FAQHZYGRMIOUNTPORBAEQWTPZ2421RI4CPV1NI' # your Foursquare ID
CLIENT_SECRET = 'LBKBBXNPCBSQ5FYLEZ5QOPJCK4X5DBYGSKBAFENP4FRQ3Y4M' # your Foursquare Secret
VERSION = '20180604'

In [54]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [55]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [56]:
toronto_venues.shape


(1616, 7)

In [57]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [58]:
#Grouping the venues by neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [59]:
print('There are {} unique venues.'.format(len(toronto_venues['Venue Category'].unique())))


There are 236 unique venues.


In [60]:
toronto_venues['Venue Category'].unique()


array(['Trail', 'Health Food Store', 'Pub', 'Park', 'Neighborhood',
       'Coffee Shop', 'Cosmetics Shop', 'Greek Restaurant',
       'Italian Restaurant', 'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Bakery',
       'Caribbean Restaurant', 'Café', 'Indian Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'American Restaurant', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Movie Theater',
       'Sandwich Place', 'Board Shop', 'Food & Drink Shop', 'Fish Market',
       'Gay Bar', 'Seafood Restaurant', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Stationery Store', 'Wine Bar', 'Thai Restaurant',
       'Coworking Space', 'Latin American Restaurant', 'Gastropub

In [61]:
grouped_venues = toronto_venues.groupby('Venue Category').count()

In [62]:
sorted_grouped_venues = grouped_venues.sort_values("Neighbourhood", axis=0, ascending=False, inplace=False, kind= "quicksort", na_position= "last")
print(sorted_grouped_venues)

                         Neighbourhood  Neighbourhood Latitude  \
Venue Category                                                   
Coffee Shop                        142                     142   
Café                                89                      89   
Restaurant                          59                      59   
Italian Restaurant                  41                      41   
Hotel                               34                      34   
...                                ...                     ...   
German Restaurant                    1                       1   
Garden Center                        1                       1   
Fruit & Vegetable Store              1                       1   
Frozen Yogurt Shop                   1                       1   
Harbor / Marina                      1                       1   

                         Neighbourhood Longitude  Venue  Venue Latitude  \
Venue Category                                                    

In [63]:
#check if the results contain any Sushi Restaurant
"Sushi Restaurant" in toronto_venues['Venue Category'].unique()

True

# Identifying the unique characteristics of each neighbourhood <a name="characteristics"></a>

In [64]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add back the neighbourhood column to dataframe
to_onehot['Neighbourhoods'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1616, 237)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Now, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [65]:
to_grouped = to_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 237)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
len(to_grouped[to_grouped["Sushi Restaurant"] > 0])


15

In [67]:
sushi_group = to_grouped[["Neighbourhoods","Sushi Restaurant"]]


In [68]:
sushi_group.head()


,Neighbourhoods,Sushi Restaurant
0,Berczy Park,0.017241
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.016129


## Using K-Means to Cluster Neighbourhoods <a name="k-means"></a>

In [69]:
from sklearn.cluster import KMeans
# set number of clusters
toclusters = 3

to_clustering = sushi_group.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 0, 2, 0, 2], dtype=int32)

In [70]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = sushi_group.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [71]:
to_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
to_merged.head()

,Neighbourhood,Sushi Restaurant,Cluster Labels
0,Berczy Park,0.017241,2
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,2
2,"Business reply mail Processing Centre, South C...",0.000000,2
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,2
4,Central Bay Street,0.016129,2


In [72]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood
to_merged = to_merged.join(toronto_venues.set_index("Neighbourhood"), on="Neighbourhood")

print(to_merged.shape)
to_merged.head()

(1616, 9)


,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,2,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017241,2,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017241,2,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017241,2,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017241,2,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [73]:
#Sorting the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1616, 9)


,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto, Harbord",0.029412,0,43.662696,-79.400049,East of Brunswick,43.665609,-79.403324,Pub
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,H Mart,43.669332,-79.386257,Grocery Store
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,Naan & Kabob,43.669005,-79.386219,Afghan Restaurant
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,7 West Cafe,43.668665,-79.386830,Café
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,Tika Tea House,43.668797,-79.386010,Bubble Tea Shop
...,...,...,...,...,...,...,...,...,...
13,"Garden District, Ryerson",0.000000,2,43.657162,-79.378937,Panago,43.658258,-79.384313,Pizza Place
13,"Garden District, Ryerson",0.000000,2,43.657162,-79.378937,College Park Area,43.659453,-79.383785,Park
13,"Garden District, Ryerson",0.000000,2,43.657162,-79.378937,Kabul Express,43.656691,-79.376643,Middle Eastern Restaurant
13,"Garden District, Ryerson",0.000000,2,43.657162,-79.378937,Victoria's Secret PINK,43.653357,-79.380673,Lingerie Store


## Plotting the Clustered Neighbourhoods <a name="clusterplot"></a>

In [74]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [75]:

# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighbourhood Latitude'], to_merged['Neighbourhood Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
display(map_clusters)

In [107]:
map_clusters.save('map_clusters.html')

## Unpacking the Clusters <a name="analyze"></a>

In [80]:
cluster0 = to_merged.loc[to_merged['Cluster Labels'] == 0]
cluster0

,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto, Harbord",0.029412,0,43.662696,-79.400049,East of Brunswick,43.665609,-79.403324,Pub
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,H Mart,43.669332,-79.386257,Grocery Store
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,Naan & Kabob,43.669005,-79.386219,Afghan Restaurant
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,7 West Cafe,43.668665,-79.386830,Café
6,Church and Wellesley,0.051948,0,43.665860,-79.383160,Tika Tea House,43.668797,-79.386010,Bubble Tea Shop
...,...,...,...,...,...,...,...,...,...
38,"University of Toronto, Harbord",0.029412,0,43.662696,-79.400049,Second Cup,43.663551,-79.401787,Café
23,"Queen's Park, Ontario Provincial Government",0.027778,0,43.662301,-79.389494,W Burger Bar,43.661331,-79.383557,Burger Joint
23,"Queen's Park, Ontario Provincial Government",0.027778,0,43.662301,-79.389494,Tokyo Grill,43.665085,-79.384707,Japanese Restaurant
16,"India Bazaar, The Beaches West",0.045455,0,43.668999,-79.315572,System Fitness,43.667171,-79.312733,Gym


In [101]:
pd.reset_option('display.max_rows', None)
cluster0.groupby('Venue Category').count()['Neighbourhood']


Venue Category
Afghan Restaurant                1
American Restaurant              2
Art Gallery                      1
Arts & Crafts Store              2
Bagel Shop                       1
                                ..
Vegetarian / Vegan Restaurant    2
Video Game Store                 1
Vietnamese Restaurant            1
Wings Joint                      2
Yoga Studio                      5
Name: Neighbourhood, Length: 102, dtype: int64

In [105]:
cluster1 = to_merged.loc[to_merged['Cluster Labels'] == 1]
cluster1

,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,"Forest Hill North & West, Forest Hill Road Park",0.25,1,43.696948,-79.411307,Oliver jewelry,43.700374,-79.407644,Jewelry Store
12,"Forest Hill North & West, Forest Hill Road Park",0.25,1,43.696948,-79.411307,Nikko Sushi Japenese Restaurant,43.700443,-79.407957,Sushi Restaurant
12,"Forest Hill North & West, Forest Hill Road Park",0.25,1,43.696948,-79.411307,Forest Hill Road Park,43.697945,-79.406605,Park
12,"Forest Hill North & West, Forest Hill Road Park",0.25,1,43.696948,-79.411307,Kay Gardner Beltline Trail,43.700726,-79.410101,Trail


In [103]:
cluster2 = to_merged.loc[to_merged['Cluster Labels'] == 2]
cluster2.shape

,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,"Regent Park, Harbourfront",0.0,2,43.654260,-79.360636,TD Canada Trust,43.652780,-79.356336,Bank
20,"Moore Park, Summerhill East",0.0,2,43.689574,-79.383160,Reds,43.692816,-79.384504,Restaurant
21,"North Toronto West, Lawrence Park",0.0,2,43.715383,-79.405678,Blo Blow Dry Bar,43.714712,-79.400227,Health & Beauty Service
21,"North Toronto West, Lawrence Park",0.0,2,43.715383,-79.405678,Pet Valu,43.714126,-79.399975,Pet Store
21,"North Toronto West, Lawrence Park",0.0,2,43.715383,-79.405678,Budget Car & Truck Rental,43.718479,-79.401165,Rental Car Location
...,...,...,...,...,...,...,...,...,...
13,"Garden District, Ryerson",0.0,2,43.657162,-79.378937,Panago,43.658258,-79.384313,Pizza Place
13,"Garden District, Ryerson",0.0,2,43.657162,-79.378937,College Park Area,43.659453,-79.383785,Park
13,"Garden District, Ryerson",0.0,2,43.657162,-79.378937,Kabul Express,43.656691,-79.376643,Middle Eastern Restaurant
13,"Garden District, Ryerson",0.0,2,43.657162,-79.378937,Victoria's Secret PINK,43.653357,-79.380673,Lingerie Store


In [106]:
cluster2.sort_values("Sushi Restaurant", axis=0, ascending=False, inplace=False, kind= "quicksort", na_position= "last")

,Neighbourhood,Sushi Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
25,"Richmond, Adelaide, King",0.021978,2,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
25,"Richmond, Adelaide, King",0.021978,2,43.650571,-79.384568,McEwan Foods,43.647160,-79.381044,Deli / Bodega
25,"Richmond, Adelaide, King",0.021978,2,43.650571,-79.384568,Prairie Girl Bakery,43.648332,-79.382305,Cupcake Shop
25,"Richmond, Adelaide, King",0.021978,2,43.650571,-79.384568,Design Exchange,43.647972,-79.380104,Art Gallery
25,"Richmond, Adelaide, King",0.021978,2,43.650571,-79.384568,SEPHORA,43.653688,-79.380120,Cosmetics Shop
...,...,...,...,...,...,...,...,...,...
7,"Commerce Court, Victoria Hotel",0.000000,2,43.648198,-79.379817,Five Guys,43.648799,-79.381681,Burger Joint
7,"Commerce Court, Victoria Hotel",0.000000,2,43.648198,-79.379817,Meridian Hall,43.646292,-79.376022,Concert Hall
7,"Commerce Court, Victoria Hotel",0.000000,2,43.648198,-79.379817,Sam James Coffee Bar (SJCB),43.647881,-79.384332,Café
7,"Commerce Court, Victoria Hotel",0.000000,2,43.648198,-79.379817,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant
